In [2]:
import os
import pandas as pd
import re
import math
import duckdb
import numpy as np
# Display all columns
pd.set_option('display.max_columns', None)

# Set max column width to a large value
pd.set_option('display.max_colwidth', None)

# Create a new DuckDB database file
con = duckdb.connect('balancer_dynamic_fee.duckdb')

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Read the CSV file line by line
file_path = 'decoded_arbitrum.csv'
lines = []

with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize dictionaries to store lines for each transaction type
transaction_dict = {}

# Function to parse fields without combining text between single quotes
def parse_line(line):
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    return fields

# Iterate over each line to categorize it based on the 6th field
for line in lines:
    fields = parse_line(line.strip())
    if len(fields) > 5:  # Ensure there are at least 6 fields after cleaning
        transaction_type = fields[5]
        if transaction_type not in transaction_dict:
            transaction_dict[transaction_type] = []
        transaction_dict[transaction_type].append(fields)

# Create separate DataFrames for each transaction type
dataframes = {}
for transaction_type, data in transaction_dict.items():
    # Determine the number of columns for this transaction type
    num_columns = max(len(fields) for fields in data)
    # Pad the data with empty strings to ensure all rows have the same number of columns
    padded_data = [fields + [''] * (num_columns - len(fields)) for fields in data]
    # Create DataFrame
    df = pd.DataFrame(padded_data, columns=[f'Field_{i+1}' for i in range(num_columns)])
    dataframes[transaction_type] = df

# Print out the names of the created DataFrames
for transaction_type in dataframes.keys():
    print(f'Created DataFrame: {transaction_type}')


Created DataFrame: PoolRegistered
Created DataFrame: TokensRegistered
Created DataFrame: PoolBalanceChanged
Created DataFrame: Swap
Created DataFrame: InternalBalanceChanged
Created DataFrame: RelayerApprovalChanged
Created DataFrame: FlashLoan
Created DataFrame: PoolBalanceManaged
Created DataFrame: AuthorizerChanged
Created DataFrame: ExternalBalanceTransfer
Created DataFrame: TokensDeregistered


In [3]:
# Function to combine fields after Field_6 and before '0x'
def combine_fields(df):
    combined_data = []
    for index, row in df.iterrows():
        combined_row = row[:6].tolist()  # Get the first six fields
        remaining_fields = row[6:].tolist()  # Get the remaining fields
        combined_field = []
        for field in remaining_fields:
            if re.match(r'^0x', field):  # Check for '0x' pattern
                break
            combined_field.append(field)
        combined_row.append(' '.join(combined_field))
        combined_data.append(combined_row + remaining_fields[len(combined_field):])
    return pd.DataFrame(combined_data, columns=[f'Field_{i+1}' for i in range(len(combined_data[0]))])

dataframes['Swap'] = combine_fields(dataframes['Swap'])
dataframes['PoolRegistered'] = combine_fields(dataframes['PoolRegistered'])
dataframes['PoolBalanceChanged'] = combine_fields(dataframes['PoolBalanceChanged'])
dataframes['PoolBalanceManaged'] = combine_fields(dataframes['PoolBalanceManaged'])

In [4]:
Swap = dataframes['Swap']
PoolRegistered = dataframes['PoolRegistered']
PoolBalanceChanged = dataframes['PoolBalanceChanged']
PoolBalanceManaged = dataframes['PoolBalanceManaged']
ExternalBalanceTransfer = dataframes['ExternalBalanceTransfer']
FlashLoan = dataframes['FlashLoan']
PoolBalanceManaged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16
0,67329900,1,3,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89L\x82\x80\x05&\xe09\x1ep\x9c\t\x83\xa5\xae\xa3q\x8b\x7fm\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03O',0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9,-106205102798,106205102798,None,None,None,None,None
1,67329900,1,4,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89L\x82\x80\x05&\xe09\x1ep\x9c\t\x83\xa5\xae\xa3q\x8b\x7fm\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03O',0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9,0,-106205102798,None,None,None,None,None
2,67329900,1,22,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89L\x82\x80\x05&\xe09\x1ep\x9c\t\x83\xa5\xae\xa3q\x8b\x7fm\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03O',0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0x0179baC7493a92AC812730a4C64A0b41B7eA0ecf,0,105992890962,None,None,None,None,None
3,67329900,1,25,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89L\x82\x80\x05&\xe09\x1ep\x9c\t\x83\xa5\xae\xa3q\x8b\x7fm\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03O',0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0x0179baC7493a92AC812730a4C64A0b41B7eA0ecf,105992890962,-105992890962,None,None,None,None,None
4,67330055,1,3,0xfff19ac713a4c2828d54fb6505d669ae6097986ed8f7a776545671d33852d56a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'[\xaer\xb7\\\xaa\xb1\xf2`\xd2\x1b\xc0(\xc60\x14\x06\x07\xd6\xe8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03P',0x84895E774E96FCBD5DE6f70AE874499aE2049409,0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8,-109967070046,109967070046,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,127141363,1,13,0x8815fc778f3e2662a2a8ca4bbd52ab4cee709c3c296e035520d2f6536c8d97a3,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'9Qe\x8e\x99\x86\xd7\x19J:5\xa1\xf5\xfe\xab\x92\xf6n\x03\xe0\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x04\xa8',0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0x0c880f6761F1af8d9Aa9C466984b80DAb9a8c9e8,112800710226815503670,-112800710226815503670,None,None,None,None,None
378,137449135,2,1,0x890076913a0b883832b569d5f39fa13a28c5fde741554e22aab3eda21049b760,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x00\x87{\x8f\xc1{negVS\xbd\x85\xc7ik\xdbv\x7f\xf7\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x04\xb9',0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0x9623063377AD1B27544C965cCd7342f7EA7e88C7,-232084022637092418774,232084022637092418774,None,None,None,None,None
379,137449135,2,2,0x890076913a0b883832b569d5f39fa13a28c5fde741554e22aab3eda21049b760,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x00\x87{\x8f\xc1{negVS\xbd\x85\xc7ik\xdbv\x7f\xf7\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x04\xb9',0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0x9623063377AD1B27544C965cCd7342f7EA7e88C7,0,-232084022637092418774,None,None,None,None,None
380,188058438,1,9,0x919c6c1c9b3d8fdb424440e3e03671e436fd5358cab1c6c647a869dc3d988d09,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'*\xe2\xba\xee\xc8\xcc\xd1`u\xd8!\x83/\xfe\xe9\x17+\xae6v\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x04\xf1',0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0xaf88d065e77c8cC2239327C5EDb3A432268e5831,0,134438378,None,None,None,None,None


In [5]:
# Function to add double quotes around the value
def byte_to_hex(byte_string):
    # Convert the string representation of the byte string to an actual byte string
    byte_string = eval(byte_string)
    # Convert the byte string to a hexadecimal string
    return byte_string.hex()

# Apply the function to the Field_7 column
PoolBalanceChanged['Field_7'] = PoolBalanceChanged['Field_7'].apply(byte_to_hex)
PoolBalanceManaged['Field_7'] = PoolBalanceManaged['Field_7'].apply(byte_to_hex)
Swap['Field_7'] = Swap['Field_7'].apply(byte_to_hex)

In [6]:
PoolBalanceManaged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16
0,67329900,1,3,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,894c82800526e0391e709c0983a5aea3718b7f6d00000000000000000000034f,0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9,-106205102798,106205102798,None,None,None,None,None
1,67329900,1,4,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,894c82800526e0391e709c0983a5aea3718b7f6d00000000000000000000034f,0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0xFd086bC7CD5C481DCC9C85ebE478A1C0b69FCbb9,0,-106205102798,None,None,None,None,None
2,67329900,1,22,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,894c82800526e0391e709c0983a5aea3718b7f6d00000000000000000000034f,0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0x0179baC7493a92AC812730a4C64A0b41B7eA0ecf,0,105992890962,None,None,None,None,None
3,67329900,1,25,0xb6dfbe7a01c2ec9dda96dbb1bc96377246f257c332428982d692a207626b411d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,894c82800526e0391e709c0983a5aea3718b7f6d00000000000000000000034f,0x3bCEBCc0d3b495cc68CD3949c791C622Ba49A9A5,0x0179baC7493a92AC812730a4C64A0b41B7eA0ecf,105992890962,-105992890962,None,None,None,None,None
4,67330055,1,3,0xfff19ac713a4c2828d54fb6505d669ae6097986ed8f7a776545671d33852d56a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,5bae72b75caab1f260d21bc028c630140607d6e8000000000000000000000350,0x84895E774E96FCBD5DE6f70AE874499aE2049409,0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8,-109967070046,109967070046,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,127141363,1,13,0x8815fc778f3e2662a2a8ca4bbd52ab4cee709c3c296e035520d2f6536c8d97a3,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,3951658e9986d7194a3a35a1f5feab92f66e03e00001000000000000000004a8,0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0x0c880f6761F1af8d9Aa9C466984b80DAb9a8c9e8,112800710226815503670,-112800710226815503670,None,None,None,None,None
378,137449135,2,1,0x890076913a0b883832b569d5f39fa13a28c5fde741554e22aab3eda21049b760,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,00877b8fc17b6e65675653bd85c7696bdb767ff70001000000000000000004b9,0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0x9623063377AD1B27544C965cCd7342f7EA7e88C7,-232084022637092418774,232084022637092418774,None,None,None,None,None
379,137449135,2,2,0x890076913a0b883832b569d5f39fa13a28c5fde741554e22aab3eda21049b760,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,00877b8fc17b6e65675653bd85c7696bdb767ff70001000000000000000004b9,0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0x9623063377AD1B27544C965cCd7342f7EA7e88C7,0,-232084022637092418774,None,None,None,None,None
380,188058438,1,9,0x919c6c1c9b3d8fdb424440e3e03671e436fd5358cab1c6c647a869dc3d988d09,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,2ae2baeec8ccd16075d821832ffee9172bae36760001000000000000000004f1,0x18cA3F1B3718984bD5d09236A59dD5Fae5C670Fa,0xaf88d065e77c8cC2239327C5EDb3A432268e5831,0,134438378,None,None,None,None,None


In [7]:
PoolBalanceChanged[['Field_9', 'Field_10', 'Field_11', 'Field_12', 'Field_13', 'Field_14']].to_csv('PoolBalanceChanged.txt')

In [8]:
import pandas as pd
import re

# Read the text file
file_path = 'PoolBalanceChanged.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Function to replace commas with dashes within square brackets
def replace_commas(text):
    return re.sub(r'\[([^\]]+)\]', lambda m: '[' + m.group(1).replace(',', '/') + ']', text)

# Apply the function to each line
modified_lines = [replace_commas(line) for line in lines]

# Save the modified lines back to a new text file (optional)
output_path = 'PoolBalanceChanged_modified.txt'
with open(output_path, 'w') as file:
    file.writelines(modified_lines)

# Create the pandas dataframe
PoolBalanceChanged_lists = pd.read_csv('PoolBalanceChanged_modified.txt')
PoolBalanceChanged_lists

,Unnamed: 0,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14
0,0,['0x6694340fc020c5E6B96567843da2df01b2CE1eb6'/'0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'],[1000000000000000000/795119],[0/0],NaN,NaN,NaN
1,1,['0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8'/'0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'],[-23943287443850709010/-64738753496544505],[58896584987846744/0],NaN,NaN,NaN
2,2,['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'/'0xa684cd057951541187f288294a1e1C2646aA2d24'],[5992954668754079/14354955094874194340],[515961742990699/0],NaN,NaN,NaN
3,3,['0x38c0a5443c7427e65A9Bf15AE746a28BB9a052cc'/'0xf581571DBcCeD3A59AaaCbf90448E7B3E1704dcD'/'0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'],[-159862836565/-162802814153/-150480016146],NaN,NaN,NaN,NaN
4,4,['0x05A131B3Cd23Be0b4F7B274B3d237E73650e543d'/'0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f'/'0x85700dC0bfD128DD0e7B9eD38496A60baC698fc8'],[0/7202751/8879896035],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
485417,485417,['0x3082CC23568eA640225c2467653dB90e9250AaA0'/'0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'],[0/318171013159994191],[0/0],NaN,NaN,NaN
485418,485418,['0x3082CC23568eA640225c2467653dB90e9250AaA0'/'0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'],[-431915392911422912745/-5813377808502169],[0/0],NaN,NaN,NaN
485419,485419,['0x3082CC23568eA640225c2467653dB90e9250AaA0'/'0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'],[48393500797696517284/654511803881250],[0/0],NaN,NaN,NaN
485420,485420,['0x3082CC23568eA640225c2467653dB90e9250AaA0'/'0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'],[0/-7363997120559253],[0/0],NaN,NaN,NaN


In [9]:

# Function to convert the string to a list of strings
def convert_to_list(cell_value):
    if isinstance(cell_value, str):
        # Remove brackets
        cell_value = cell_value.strip('[]')
        # Split by '/'
        split_values = cell_value.split('/')
        # Convert to list of strings
        return [str(value) for value in split_values]
    else:
        return cell_value

# Apply the function to all columns
for column in ['Field_9', 'Field_10', 'Field_11']:
    PoolBalanceChanged_lists[column] = PoolBalanceChanged_lists[column].apply(convert_to_list)

PoolBalanceChanged_lists

,Unnamed: 0,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14
0,0,"['0x6694340fc020c5E6B96567843da2df01b2CE1eb6', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[1000000000000000000, 795119]","[0, 0]",NaN,NaN,NaN
1,1,"['0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-23943287443850709010, -64738753496544505]","[58896584987846744, 0]",NaN,NaN,NaN
2,2,"['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', '0xa684cd057951541187f288294a1e1C2646aA2d24']","[5992954668754079, 14354955094874194340]","[515961742990699, 0]",NaN,NaN,NaN
3,3,"['0x38c0a5443c7427e65A9Bf15AE746a28BB9a052cc', '0xf581571DBcCeD3A59AaaCbf90448E7B3E1704dcD', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[-159862836565, -162802814153, -150480016146]",NaN,NaN,NaN,NaN
4,4,"['0x05A131B3Cd23Be0b4F7B274B3d237E73650e543d', '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f', '0x85700dC0bfD128DD0e7B9eD38496A60baC698fc8']","[0, 7202751, 8879896035]",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
485417,485417,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, 318171013159994191]","[0, 0]",NaN,NaN,NaN
485418,485418,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-431915392911422912745, -5813377808502169]","[0, 0]",NaN,NaN,NaN
485419,485419,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[48393500797696517284, 654511803881250]","[0, 0]",NaN,NaN,NaN
485420,485420,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, -7363997120559253]","[0, 0]",NaN,NaN,NaN


In [10]:
PoolBalanceChanged['Field_9'] = PoolBalanceChanged_lists['Field_9']
PoolBalanceChanged['Field_10'] = PoolBalanceChanged_lists['Field_10']
PoolBalanceChanged['Field_11'] = PoolBalanceChanged_lists['Field_11']
PoolBalanceChanged = PoolBalanceChanged[[f'Field_{i}' for i in range(1, 12)]]
PoolBalanceChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11
0,8114621,0,6,0x08fddf4b4db47698f62940d9ca5445c3630d5b5e8c0412eadfb3f1c51caacbcb,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,3a4c6d2404b5eb14915041e01f63200a82f4a343000200000000000000000065,0xeC576a26335dE1C360d2FC9A68cbA6BA37af4a13,"['0x6694340fc020c5E6B96567843da2df01b2CE1eb6', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[1000000000000000000, 795119]","[0, 0]"
1,8115562,0,4,0x04a6d804da1399d6ce117a82dff255616ec2f2a23141289f12ed7a9efed63022,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cc65a812ce382ab909a11e434dbf75b34f1cc59d000200000000000000000001,0x0eec7f3c79C0AdaD5Cd90875b7ef199335bA0100,"['0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-23943287443850709010, -64738753496544505]","[58896584987846744, 0]"
2,8116923,0,5,0x9fc02fa487ebb7792d89f840f0d31e885407e458884f63f2a7e455ec4f5f8efd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,c61ff48f94d801c1ceface0289085197b5ec44f000020000000000000000004d,0x88F91bDa415526c675E8f33B73469dccB3F782f0,"['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', '0xa684cd057951541187f288294a1e1C2646aA2d24']","[5992954668754079, 14354955094874194340]","[515961742990699, 0]"
3,8117217,0,5,0x7527b4754b2ec5c80155d0efa9689a9fcd8e6609d2c51c5efca723dc2aa1183a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,6ee86e032173716a41818e6d6d320a752176d69700010000000000000000001c,0xf5830FF9B625b60131D82530c51c9e294a9f7478,"['0x38c0a5443c7427e65A9Bf15AE746a28BB9a052cc', '0xf581571DBcCeD3A59AaaCbf90448E7B3E1704dcD', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[-159862836565, -162802814153, -150480016146]",NaN
4,8117605,0,6,0xedc076d600f1157b93dc29d217c13e349dcfd92040408473e18f306e0b604e30,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cf3ae4b9235b1c203457e472a011c1203a2fde93000100000000000000000019,0xFBA206E73709F614E5a85afF27A98692d4F3c579,"['0x05A131B3Cd23Be0b4F7B274B3d237E73650e543d', '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f', '0x85700dC0bfD128DD0e7B9eD38496A60baC698fc8']","[0, 7202751, 8879896035]",NaN
...,...,...,...,...,...,...,...,...,...,...,...
485417,217404804,1,199,0xcf2bf44efdc928991810c0b292ef9a48d813f7815b29768fdd19a4e65a53386f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, 318171013159994191]","[0, 0]"
485418,217407061,4,12,0xff536e047bbda86749cdf0dd02d0a5cb20f94540a44d93d17a9ecc79f4b3de34,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xEF29bd856d18410A041a8a5d419568497c65536E,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-431915392911422912745, -5813377808502169]","[0, 0]"
485419,217407689,2,13,0xbf715c0fa7b5a93a88b8cd2e34daea9e4d00f14d8c4f145148892c44793156e6,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[48393500797696517284, 654511803881250]","[0, 0]"
485420,217411692,4,7,0x2bc4d1f67c9d52607c92a5a84a2094a5455c779457b5c873a299c8f5c41e1f21,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xb3ecf7daDd14a157975c2Bd445306Cfe428Dd668,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, -7363997120559253]","[0, 0]"


In [11]:
PoolBalanceChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11
0,8114621,0,6,0x08fddf4b4db47698f62940d9ca5445c3630d5b5e8c0412eadfb3f1c51caacbcb,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,3a4c6d2404b5eb14915041e01f63200a82f4a343000200000000000000000065,0xeC576a26335dE1C360d2FC9A68cbA6BA37af4a13,"['0x6694340fc020c5E6B96567843da2df01b2CE1eb6', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[1000000000000000000, 795119]","[0, 0]"
1,8115562,0,4,0x04a6d804da1399d6ce117a82dff255616ec2f2a23141289f12ed7a9efed63022,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cc65a812ce382ab909a11e434dbf75b34f1cc59d000200000000000000000001,0x0eec7f3c79C0AdaD5Cd90875b7ef199335bA0100,"['0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-23943287443850709010, -64738753496544505]","[58896584987846744, 0]"
2,8116923,0,5,0x9fc02fa487ebb7792d89f840f0d31e885407e458884f63f2a7e455ec4f5f8efd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,c61ff48f94d801c1ceface0289085197b5ec44f000020000000000000000004d,0x88F91bDa415526c675E8f33B73469dccB3F782f0,"['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', '0xa684cd057951541187f288294a1e1C2646aA2d24']","[5992954668754079, 14354955094874194340]","[515961742990699, 0]"
3,8117217,0,5,0x7527b4754b2ec5c80155d0efa9689a9fcd8e6609d2c51c5efca723dc2aa1183a,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,6ee86e032173716a41818e6d6d320a752176d69700010000000000000000001c,0xf5830FF9B625b60131D82530c51c9e294a9f7478,"['0x38c0a5443c7427e65A9Bf15AE746a28BB9a052cc', '0xf581571DBcCeD3A59AaaCbf90448E7B3E1704dcD', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[-159862836565, -162802814153, -150480016146]",NaN
4,8117605,0,6,0xedc076d600f1157b93dc29d217c13e349dcfd92040408473e18f306e0b604e30,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cf3ae4b9235b1c203457e472a011c1203a2fde93000100000000000000000019,0xFBA206E73709F614E5a85afF27A98692d4F3c579,"['0x05A131B3Cd23Be0b4F7B274B3d237E73650e543d', '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f', '0x85700dC0bfD128DD0e7B9eD38496A60baC698fc8']","[0, 7202751, 8879896035]",NaN
...,...,...,...,...,...,...,...,...,...,...,...
485417,217404804,1,199,0xcf2bf44efdc928991810c0b292ef9a48d813f7815b29768fdd19a4e65a53386f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, 318171013159994191]","[0, 0]"
485418,217407061,4,12,0xff536e047bbda86749cdf0dd02d0a5cb20f94540a44d93d17a9ecc79f4b3de34,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xEF29bd856d18410A041a8a5d419568497c65536E,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-431915392911422912745, -5813377808502169]","[0, 0]"
485419,217407689,2,13,0xbf715c0fa7b5a93a88b8cd2e34daea9e4d00f14d8c4f145148892c44793156e6,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[48393500797696517284, 654511803881250]","[0, 0]"
485420,217411692,4,7,0x2bc4d1f67c9d52607c92a5a84a2094a5455c779457b5c873a299c8f5c41e1f21,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xb3ecf7daDd14a157975c2Bd445306Cfe428Dd668,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, -7363997120559253]","[0, 0]"


In [12]:
# Select and rename specific columns
selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'transaction_sender', 'Field_9': 'tokens', 'Field_10': 'delta', 'Field_11': 'protocol_fee'}
PoolBalanceChanged = PoolBalanceChanged[list(selected_columns.keys())].rename(columns=selected_columns)

selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'transaction_sender', 'Field_9': 'token', 'Field_10': 'cash_delta'}
PoolBalanceManaged = PoolBalanceManaged[list(selected_columns.keys())].rename(columns=selected_columns)

selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'token_in', 'Field_9': 'token_out', 'Field_10': 'amount_in', 'Field_11': 'amount_out'}
Swap = Swap[list(selected_columns.keys())].rename(columns=selected_columns)

PoolBalanceChanged

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
0,8114621,6,0x08fddf4b4db47698f62940d9ca5445c3630d5b5e8c0412eadfb3f1c51caacbcb,PoolBalanceChanged,3a4c6d2404b5eb14915041e01f63200a82f4a343000200000000000000000065,0xeC576a26335dE1C360d2FC9A68cbA6BA37af4a13,"['0x6694340fc020c5E6B96567843da2df01b2CE1eb6', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[1000000000000000000, 795119]","[0, 0]"
1,8115562,4,0x04a6d804da1399d6ce117a82dff255616ec2f2a23141289f12ed7a9efed63022,PoolBalanceChanged,cc65a812ce382ab909a11e434dbf75b34f1cc59d000200000000000000000001,0x0eec7f3c79C0AdaD5Cd90875b7ef199335bA0100,"['0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-23943287443850709010, -64738753496544505]","[58896584987846744, 0]"
2,8116923,5,0x9fc02fa487ebb7792d89f840f0d31e885407e458884f63f2a7e455ec4f5f8efd,PoolBalanceChanged,c61ff48f94d801c1ceface0289085197b5ec44f000020000000000000000004d,0x88F91bDa415526c675E8f33B73469dccB3F782f0,"['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', '0xa684cd057951541187f288294a1e1C2646aA2d24']","[5992954668754079, 14354955094874194340]","[515961742990699, 0]"
3,8117217,5,0x7527b4754b2ec5c80155d0efa9689a9fcd8e6609d2c51c5efca723dc2aa1183a,PoolBalanceChanged,6ee86e032173716a41818e6d6d320a752176d69700010000000000000000001c,0xf5830FF9B625b60131D82530c51c9e294a9f7478,"['0x38c0a5443c7427e65A9Bf15AE746a28BB9a052cc', '0xf581571DBcCeD3A59AaaCbf90448E7B3E1704dcD', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[-159862836565, -162802814153, -150480016146]",NaN
4,8117605,6,0xedc076d600f1157b93dc29d217c13e349dcfd92040408473e18f306e0b604e30,PoolBalanceChanged,cf3ae4b9235b1c203457e472a011c1203a2fde93000100000000000000000019,0xFBA206E73709F614E5a85afF27A98692d4F3c579,"['0x05A131B3Cd23Be0b4F7B274B3d237E73650e543d', '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f', '0x85700dC0bfD128DD0e7B9eD38496A60baC698fc8']","[0, 7202751, 8879896035]",NaN
...,...,...,...,...,...,...,...,...,...
485417,217404804,199,0xcf2bf44efdc928991810c0b292ef9a48d813f7815b29768fdd19a4e65a53386f,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, 318171013159994191]","[0, 0]"
485418,217407061,12,0xff536e047bbda86749cdf0dd02d0a5cb20f94540a44d93d17a9ecc79f4b3de34,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xEF29bd856d18410A041a8a5d419568497c65536E,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-431915392911422912745, -5813377808502169]","[0, 0]"
485419,217407689,13,0xbf715c0fa7b5a93a88b8cd2e34daea9e4d00f14d8c4f145148892c44793156e6,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[48393500797696517284, 654511803881250]","[0, 0]"
485420,217411692,7,0x2bc4d1f67c9d52607c92a5a84a2094a5455c779457b5c873a299c8f5c41e1f21,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xb3ecf7daDd14a157975c2Bd445306Cfe428Dd668,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, -7363997120559253]","[0, 0]"


In [13]:
PoolBalanceChanged

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
0,8114621,6,0x08fddf4b4db47698f62940d9ca5445c3630d5b5e8c0412eadfb3f1c51caacbcb,PoolBalanceChanged,3a4c6d2404b5eb14915041e01f63200a82f4a343000200000000000000000065,0xeC576a26335dE1C360d2FC9A68cbA6BA37af4a13,"['0x6694340fc020c5E6B96567843da2df01b2CE1eb6', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[1000000000000000000, 795119]","[0, 0]"
1,8115562,4,0x04a6d804da1399d6ce117a82dff255616ec2f2a23141289f12ed7a9efed63022,PoolBalanceChanged,cc65a812ce382ab909a11e434dbf75b34f1cc59d000200000000000000000001,0x0eec7f3c79C0AdaD5Cd90875b7ef199335bA0100,"['0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-23943287443850709010, -64738753496544505]","[58896584987846744, 0]"
2,8116923,5,0x9fc02fa487ebb7792d89f840f0d31e885407e458884f63f2a7e455ec4f5f8efd,PoolBalanceChanged,c61ff48f94d801c1ceface0289085197b5ec44f000020000000000000000004d,0x88F91bDa415526c675E8f33B73469dccB3F782f0,"['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', '0xa684cd057951541187f288294a1e1C2646aA2d24']","[5992954668754079, 14354955094874194340]","[515961742990699, 0]"
3,8117217,5,0x7527b4754b2ec5c80155d0efa9689a9fcd8e6609d2c51c5efca723dc2aa1183a,PoolBalanceChanged,6ee86e032173716a41818e6d6d320a752176d69700010000000000000000001c,0xf5830FF9B625b60131D82530c51c9e294a9f7478,"['0x38c0a5443c7427e65A9Bf15AE746a28BB9a052cc', '0xf581571DBcCeD3A59AaaCbf90448E7B3E1704dcD', '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8']","[-159862836565, -162802814153, -150480016146]",NaN
4,8117605,6,0xedc076d600f1157b93dc29d217c13e349dcfd92040408473e18f306e0b604e30,PoolBalanceChanged,cf3ae4b9235b1c203457e472a011c1203a2fde93000100000000000000000019,0xFBA206E73709F614E5a85afF27A98692d4F3c579,"['0x05A131B3Cd23Be0b4F7B274B3d237E73650e543d', '0x2f2a2543B76A4166549F7aaB2e75Bef0aefC5B0f', '0x85700dC0bfD128DD0e7B9eD38496A60baC698fc8']","[0, 7202751, 8879896035]",NaN
...,...,...,...,...,...,...,...,...,...
485417,217404804,199,0xcf2bf44efdc928991810c0b292ef9a48d813f7815b29768fdd19a4e65a53386f,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, 318171013159994191]","[0, 0]"
485418,217407061,12,0xff536e047bbda86749cdf0dd02d0a5cb20f94540a44d93d17a9ecc79f4b3de34,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xEF29bd856d18410A041a8a5d419568497c65536E,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[-431915392911422912745, -5813377808502169]","[0, 0]"
485419,217407689,13,0xbf715c0fa7b5a93a88b8cd2e34daea9e4d00f14d8c4f145148892c44793156e6,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xfC05ec21b106E0c1e035Ec4718C1394f098FBb57,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[48393500797696517284, 654511803881250]","[0, 0]"
485420,217411692,7,0x2bc4d1f67c9d52607c92a5a84a2094a5455c779457b5c873a299c8f5c41e1f21,PoolBalanceChanged,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0xb3ecf7daDd14a157975c2Bd445306Cfe428Dd668,"['0x3082CC23568eA640225c2467653dB90e9250AaA0', '0x82aF49447D8a07e3bd95BD0d56f35241523fBab1']","[0, -7363997120559253]","[0, 0]"


In [15]:
con.execute('CREATE TABLE arbitrum_swap AS SELECT * FROM Swap').fetchdf()
con.execute('CREATE TABLE arbitrum_poolregistered AS SELECT * FROM PoolRegistered').fetchdf()
con.execute('CREATE TABLE arbitrum_poolbalancechanged AS SELECT * FROM PoolBalanceChanged').fetchdf()
con.execute('CREATE TABLE arbitrum_externalbalancetransfer AS SELECT * FROM ExternalBalanceTransfer').fetchdf()
con.execute('CREATE TABLE arbitrum_flashloan AS SELECT * FROM FlashLoan').fetchdf()
con.execute('CREATE TABLE arbitrum_poolbalancemanaged AS SELECT * FROM PoolBalanceManaged').fetchdf()

# Commit the changes
con.commit()

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [21]:
PoolBalanceChanged[PoolBalanceChanged['pool_id']== '32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd']  

,block_number,log_index,transaction_hash,event,pool_id,token_in,token_out,amount_in,amount_out
2345154,71270649,0,0xd725f5dbf694315770d1da6f1acd25a2f06fcecaed3f1197b40fd056e02a2580,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,10528254411815188917223,2100000000000000000
2345194,71271430,0,0xb60b8370409d08fc587c9f4d8660bb11b3b80738433a75f09a03c79a0a0664ca,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,13714394535418507590564,2700000000000000000
2345618,71288605,11,0x812cc5418d8ed2ccaf91b751813384ad83f710a56a3f97867faddfa78b417dad,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,14087379771733187515673,3054963401666107348
2345651,71288951,12,0xc8cb16c4c55d45c84c4dd1341a5231267b63637e80f3d9608d2b2bfd0e53ecee,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,935523898204920928820,201254439664853450
2345680,71290506,7,0x2a4b5aa624f8ae22cc9ae5aaccd6c238f07992dbe912b4778e5132a2b02af1a8,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,49236910000000000000000,10340628709558429424
...,...,...,...,...,...,...,...,...,...
4824176,217384881,3,0xbf826b2c2a1fc2bf6fb68406dcd3631cfaa4722030735d1a77ad4bfeeae0bc92,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,974995791722531279665,52299344038712613
4824177,217384905,0,0x7af5f5f4f5aebc585cddc3d4491472e960b99d046443ad5ea87729489970c0b0,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,6424636866225083967443,344582840077818776
4824178,217384917,42,0xdf5249ca4c8e633cd083dfc23917cd081cb093c0f877b593622e764aaf87e2f6,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,10922420375273124935539,585666440755771298
4824182,217385034,2,0xa479a7b365a27b196357490a086593c24e90298ae81049e515b889003a0caada,Swap,32df62dc3aed20d62241930520e665dc181658410002000000000000000003bd,0x3082CC23568eA640225c2467653dB90e9250AaA0,0x82aF49447D8a07e3bd95BD0d56f35241523fBab1,6103985280153993347072,327215119435466839
